In [7]:
def gzExtract(file):
    import gzip
    import shutil
    with gzip.open(file, 'rb') as f_in:
        with open(file[:-3], 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    return file[:-3]

In [ ]:
# create a funtion to downlaod all datsets from the given urls list and save them in ./Dataset directory
import os
import requests
import pandas as pd
import numpy as np
import gzip 
import shutil

def download_datasets():   
    # CICIDS2017 Dataset
    !kaggle datasets download -d chethuhn/network-intrusion-dataset -- unzip -o -d ./Dataset/CICIDS2017
    #UNSW-NB15 Dataset
    !kaggle datasets download -d mrwellsdavid/unsw-nb15 -- unzip -o -d ./Dataset/UNSW-NB15
    #DARPA Intrusion Detection Evaluation Dataset, dirct download link, days 1,2,3
    url = 'https://archive.ll.mit.edu/ideval/data/1999/training/week2/monday/inside.tcpdump.gz'
    r = requests.get(url, allow_redirects=True)
    open('./Dataset/DARPA/inside_1.tcpdump.gz', 'wb').write(r.content)
    gzExtract("./Dataset/DARPA/inside_1.tcpdump.gz")

    url = 'https://archive.ll.mit.edu/ideval/data/1999/training/week2/tuesday/inside.tcpdump.gz'
    r = requests.get(url, allow_redirects=True)
    open('./Dataset/DARPA/inside_2.tcpdump.gz', 'wb').write(r.content)
    gzExtract("./Dataset/DARPA/inside_2.tcpdump.gz")

    url = 'https://archive.ll.mit.edu/ideval/data/1999/training/week2/wednesday/inside.tcpdump.gz'
    r = requests.get(url, allow_redirects=True)
    open('./Dataset/DARPA/inside_3.tcpdump.gz', 'wb').write(r.content)
    gzExtract("./Dataset/DARPA/inside_3.tcpdump.gz")
    #--------------------------------------------------------------------------------------#
    



    


In [6]:
import gzip 
import shutil https://archive.ll.mit.edu/ideval/data/1999/training/week2/tuesday/inside.tcpdump.gz https://archive.ll.mit.edu/ideval/data/1999/training/week2/monday/inside.tcpdump.gz
# extract the .gz file
with gzip.open('inside.tcpdump.gz', 'rb') as f_in:
    with open('instde.tcpdump', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [4]:
%pip install tar

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tar (from versions: none)
ERROR: No matching distribution found for tar
